#### Importar libs

In [1]:
import pandas as pd
import requests
from io import BytesIO
from datetime import datetime
from zoneinfo import ZoneInfo
import sys

# --- Configuração dos Módulos ---
sys.path.insert(0, r'C:\Scripts\modules_azure\database')
sys.path.insert(0, r'C:\Scripts\modules_azure\parameters')

from connection_azure import Connect
from azure_loader import AzureLoader

# --- Configurações Globais ---
SCHEMA_DEFAULT = "dbo"
TZ_BRASILIA = ZoneInfo('America/Sao_Paulo')

#### Atualizar o ROA e os volumes detalhados

In [2]:
try:
    hist = AzureLoader.ler_tabela("previa_receita_nova", schema=SCHEMA_DEFAULT)
    
    # Define o mês atual (YYYY-MM) para remover do histórico e substituir pelo novo
    mes_atual_str = datetime.now(TZ_BRASILIA).strftime("%Y-%m")
    
    # Remove dados do mês atual para evitar duplicação (Reload incremental do mês)
    if not hist.empty:
        # Garante que a coluna Data seja string YYYY-MM para comparação
        # Se estiver como datetime no banco, converte
        if pd.api.types.is_datetime64_any_dtype(hist['Data']):
            hist['Data'] = hist['Data'].dt.strftime("%Y-%m")
            
        hist = hist[hist['Data'] != mes_atual_str]
        
    print(f"   -> Histórico carregado. Dados de {mes_atual_str} removidos para atualização.")

except Exception as e:
    print(f"   -> Erro ao ler histórico: {e}")
    hist = pd.DataFrame()

[AzureLoader] Lendo: dbo.previa_receita_nova...
   -> Histórico carregado. Dados de 2026-02 removidos para atualização.


In [3]:
def load_previas(advisor_name, link: str):
    try:
        response = requests.get(link, params={"downloadformat": "excel"}, timeout=15)
        response.raise_for_status() # Garante que não foi 404
        
        df = pd.read_excel(BytesIO(response.content), sheet_name='Meta')
        
        # Validação básica de colunas
        if len(df.columns) < 5:
            print(f"   [AVISO] Arquivo de {advisor_name} parece ter colunas insuficientes.")
            return pd.DataFrame()

        # Mapeamento por posição (Mantendo lógica original)
        df.rename(columns={
            df.columns[0]: "Categoria - Acompanhamento Next",
            df.columns[1]: "META - ROA",
            df.columns[2]: "REALIZADO - ROA",
            df.columns[4]: "REALIZADO - VOLUME"
        }, inplace=True)

        df['META - VOLUME'] = 0.0
        
        # Filtros
        if "Categoria - Acompanhamento Next" in df.columns:
            df = df[df['Categoria - Acompanhamento Next'] != "TOTAL"]
        
        df['Assessor'] = advisor_name
        
        colunas_ordem = [
            'Categoria - Acompanhamento Next', 'META - VOLUME', 'REALIZADO - VOLUME', 
            'META - ROA', 'REALIZADO - ROA', 'Assessor'
        ]
        
        # Garante que só retorna colunas que existem
        cols_existentes = [c for c in colunas_ordem if c in df.columns]
        return df[cols_existentes]

    except Exception as e:
        print(f"   [ERRO] Falha ao baixar/ler {advisor_name}: {e}")
        return pd.DataFrame()

# Lista de Links
assessores_links = [
    ("RODRIGO DE MELLO D’ELIA", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQBVuGicHybdRrC4d1MtFO8vAbY4Kw4m4_8gNo8EKu3BN4I?download=1"),
    ("CAIC ZEM GOMES", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQByqpoVZXN3TYdcUlFmYqE9Af8AsDkk6umaNM26wLfQlo4?download=1"),
    ("FERNANDO DOMINGUES DA SILVA", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQAFP5cFxU9QRL_OWClG-BJdAT3Wvk18_VoypIF3CxIpQYY?download=1"),
    ("SAADALLAH JOSE ASSAD", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQDjXAOoHHxgQYo_aeQQdXz4AXzuH0TotDouMPG_NNH4H-4?download=1"),
    ("PAULO ROBERTO FARIA SILVA", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQA6mQn9Z8lwTL7HfLdw1UzSAbCoa8qOg03wNZvbJyDoIaw?download=1"),
    ("MARCOS SOARES PEREIRA FILHO", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQAtEitVDNlnT4zWgSIonhfzAQlTKUlOX3xzRRYfgcoHrZE?download=1"),
    ("RENAN BENTO DA SILVA", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQCr2UivmbvdQ4EIlpS8fcTfAVmuuqLXeLhqroDonRcYXDQ?download=1"),
    ("ROSANA APARECIDA PAVANI DA SILVA", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQDmCPZulmfjTYtC94MGxHh1ARzYBjyg9gWNL9v2U--M0CQ?download=1"),
    ("RAFAEL PASOLD MEDEIROS", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQBVpoNJYZ8NSY9rnRotpPSPAXlir-PVsE7SItom19ZhijI?download=1"),
    ("FELIPE AUGUSTO CARDOSO", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQCy_TXmqwgLSL2M6iwrcsp0AWNG5plDMI-xPRY363bX2dA?download=1"),
    ("GUILHERME DE LUCCA BERTELONI", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQCZlPDCr4lsTpfTGlsnjvQiAT0lnyonDCFpCKlASYnfES4?download=1"),
    ("IZADORA VILLELA FREITAS", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQD4lZoreFnUQYKgxi0ibLONAV1CwfMLplcscrKNd6ZJPis?download=1"),
    ("VITOR OLIVEIRA DOS REIS", "https://netorg18892072-my.sharepoint.com/:x:/g/personal/joao_aquino_atriacm_com_br/IQAmZYnDo_EeToIhGV7QRgYJAb6YYoMOMaWaAEadD6WOS4I?download=1")
]

lista_dfs = []
for nome, link in assessores_links:
    df_temp = load_previas(nome, link)
    if not df_temp.empty:
        lista_dfs.append(df_temp)
        print(f"   [OK] {nome}")

if not lista_dfs:
    print("Nenhum dado coletado. Encerrando.")
    sys.exit()

# Consolida
previa_receita = pd.concat(lista_dfs, ignore_index=True)

# Adiciona Data/Hora
previa_receita['Data'] = datetime.now(TZ_BRASILIA).strftime("%Y-%m")
previa_receita['Hora Atualizado'] = datetime.now(TZ_BRASILIA).strftime("%Y-%m-%d %H:%M:%S")

   [OK] RODRIGO DE MELLO D’ELIA
   [OK] CAIC ZEM GOMES
   [OK] FERNANDO DOMINGUES DA SILVA
   [OK] SAADALLAH JOSE ASSAD
   [OK] PAULO ROBERTO FARIA SILVA
   [OK] MARCOS SOARES PEREIRA FILHO
   [OK] RENAN BENTO DA SILVA
   [OK] ROSANA APARECIDA PAVANI DA SILVA
   [OK] RAFAEL PASOLD MEDEIROS
   [OK] FELIPE AUGUSTO CARDOSO
   [OK] GUILHERME DE LUCCA BERTELONI
   [OK] IZADORA VILLELA FREITAS
   [OK] VITOR OLIVEIRA DOS REIS


In [4]:
# Concatena histórico limpo + dados novos
previa_final_detalhada = pd.concat([hist, previa_receita], axis=0)

# Tratamento de Nulos
previa_final_detalhada.loc[previa_final_detalhada['META - VOLUME'] == '-', 'META - VOLUME'] = 0
previa_final_detalhada.fillna(0, inplace=True)

# Upload Seguro
AzureLoader.enviar_df(
    df=previa_final_detalhada,
    nome_tabela="previa_receita_nova",
    if_exists="replace",
    schema=SCHEMA_DEFAULT
)

[AzureLoader] Subindo tabela: dbo.previa_receita_nova (742 linhas)...
[AzureLoader] Chunksize calculado: 261 linhas/lote (Colunas: 8).
[AzureLoader] Concluido: previa_receita_nova atualizada.


In [5]:
# Agrupa dados novos
cols_agg = ["Assessor", "Data", "META - ROA", "REALIZADO - ROA"]
# Garante que colunas existem
cols_agg = [c for c in cols_agg if c in previa_receita.columns]
previa_receita_assessor = previa_receita[cols_agg].groupby(['Assessor', 'Data']).sum().reset_index()

# Padroniza data para dia 01 do mês (formato datetime)
previa_receita_assessor['Data'] = pd.to_datetime(previa_receita_assessor['Data'], format="%Y-%m")

# Carrega histórico agregado
historico_agg = AzureLoader.ler_tabela("previa_receita_assessor_historico", schema=SCHEMA_DEFAULT)

if not historico_agg.empty:
    historico_agg['Data'] = pd.to_datetime(historico_agg['Data'])
    
    # Data de corte: dia 1 do mês atual
    data_corte = datetime.now(TZ_BRASILIA).replace(day=1, hour=0, minute=0, second=0, microsecond=0).replace(tzinfo=None)
    
    # Remove mês atual do histórico
    historico_agg = historico_agg[historico_agg['Data'] != data_corte]
    
    # Concatena
    previa_final_agg = pd.concat([historico_agg, previa_receita_assessor], axis=0, ignore_index=True)
else:
    previa_final_agg = previa_receita_assessor

[AzureLoader] Lendo: dbo.previa_receita_assessor_historico...


In [6]:
AzureLoader.enviar_df(
    df=previa_final_agg,
    nome_tabela="previa_receita_assessor_historico",
    if_exists="replace",
    schema=SCHEMA_DEFAULT
)

[AzureLoader] Subindo tabela: dbo.previa_receita_assessor_historico (127 linhas)...
[AzureLoader] Chunksize calculado: 522 linhas/lote (Colunas: 4).
[AzureLoader] Concluido: previa_receita_assessor_historico atualizada.
